In [3]:
import requests
import pandas as pd
from nba_api.live.nba.endpoints import scoreboard
from nba_api.live.nba.endpoints import boxscore
from datetime import datetime
from datetime import timedelta
import pytz

In [4]:
stat_categories = ['jerseyNum', 'position', 'name', 'minutes',
                   'points', 'reboundsTotal', 'assists', 'steals', 'blocks']
sort_by = ['points']
asc_list = [False]

In [20]:
def get_daily_player_data(date=None):
    board = scoreboard.ScoreBoard()
    timezone = pytz.timezone('EST')
    now = datetime.now(tz=pytz.timezone('EST'))
    id_list = []
    start_times = []
    away_list = []
    home_list = []

    if date == None:
        games = board.games.get_dict()

        game_date = board.score_board_date
        print('Games Taking Place Today - {}\n \
        -----------------------------------------------------------------------------'.format(game_date))

        for game in games:
            id = game['gameId']
            away = game['awayTeam']['teamTricode']
            home = game['homeTeam']['teamTricode']
            start = datetime.strptime(
                game['gameTimeUTC'], "%Y-%m-%dT%H:%M:%S%z")
            start = start.astimezone(timezone)
            id_list.append(id)
            away_list.append(away)
            home_list.append(home)
            start_times.append(start)

            print('Game ID {}: {} at {} - {} start'.format(id,
                  away, home, start.strftime("%H:%M")))

        daily_stats = []
        for i, game_id in enumerate(id_list):
            if start_times[i] < now:
                box = boxscore.BoxScore(game_id)
                away_df = pd.DataFrame(box.away_team_player_stats.get_dict())
                home_df = pd.DataFrame(box.home_team_player_stats.get_dict())
                away_df_stats = away_df.join(pd.DataFrame(
                    (away_df.pop('statistics').values.tolist())))
                away_df_stats['team'] = away_list[i]
                home_df_stats = home_df.join(pd.DataFrame(
                    (home_df.pop('statistics').values.tolist())))
                home_df_stats['team'] = home_list[i]
                daily_stats.append(away_df_stats)
                daily_stats.append(home_df_stats)

        daily_stats_df = pd.concat(daily_stats)

        return daily_stats_df

In [21]:
get_daily_player_data()[['name','points']]

Games Taking Place Today - 2022-01-15
         -----------------------------------------------------------------------------
Game ID 0022100649: LAL at DEN - 21:00 start
Game ID 0022100646: CLE at OKC - 20:00 start
Game ID 0022100648: LAC at SAS - 20:30 start
Game ID 0022100647: CHI at BOS - 20:30 start
Game ID 0022100645: PHI at MIA - 20:00 start
Game ID 0022100644: NOP at BKN - 19:30 start
Game ID 0022100643: NYK at ATL - 19:30 start
Game ID 0022100641: TOR at MIL - 18:30 start
Game ID 0022100642: POR at WAS - 19:00 start
Game ID 0022100650: ORL at DAL - 21:30 start


,name,points
0,Malik Monk,4
1,LeBron James,8
2,Dwight Howard,11
3,Avery Bradley,3
4,Russell Westbrook,2
...,...,...
11,Tim Hardaway Jr.,0
12,Boban Marjanovic,0
13,Frank Ntilikina,0
14,Dwight Powell,0
